In [3]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(openai_api_base= "http://192.168.4.46:1234/v1",
    openai_api_key=key,
    model_name="meta-llama-3.1-8b-instruct"
)

In [6]:
import pandas as pd
def extract_metadata(df):
    metadata = {}

    # Number of columns
    metadata['Number of Columns'] = df.shape[1]

    # Column names
    metadata['Schema'] = df.columns.tolist()

    # Data types of each column
    metadata['Data Types'] = str(df.dtypes)

    # Summary statistics
    metadata['Sample'] = df.head(1).to_dict(orient="records")

    return metadata

In [7]:
df = pd.read_csv("mtcars.csv")
metadata = extract_metadata(df)

In [8]:
metadata

{'Number of Columns': 12,
 'Schema': ['model',
  'mpg',
  'cyl',
  'disp',
  'hp',
  'drat',
  'wt',
  'qsec',
  'vs',
  'am',
  'gear',
  'carb'],
 'Data Types': 'model     object\nmpg      float64\ncyl        int64\ndisp     float64\nhp         int64\ndrat     float64\nwt       float64\nqsec     float64\nvs         int64\nam         int64\ngear       int64\ncarb       int64\ndtype: object',
 'Sample': [{'model': 'Mazda RX4',
   'mpg': 21.0,
   'cyl': 6,
   'disp': 160.0,
   'hp': 110,
   'drat': 3.9,
   'wt': 2.62,
   'qsec': 16.46,
   'vs': 0,
   'am': 1,
   'gear': 4,
   'carb': 4}]}

In [9]:
prompt_template = '''
Assistant is an AI model that takes in metadata from a dataset 
and suggests charts to use to visualise that data.

New Input: Suggest 2 charts to visualise data from a dataset with the following metadata. 


SCHEMA:

-------- 

{schema}

DATA TYPES: 

-------- 

{data_types}

SAMPLE: 

-------- 

{sample}

'''.format(schema = metadata["Schema"], data_types = metadata["Data Types"], sample=metadata["Sample"])


In [11]:
llm.invoke(prompt_template).content

'Based on the metadata provided, here are two chart suggestions to visualize the data:\n\n**Chart 1: Scatter Plot of MPG vs Displacement**\n\n* X-axis: displacement (disp)\n* Y-axis: miles per gallon (mpg)\n* Color by: model\n* This chart can help identify any relationships between engine size and fuel efficiency. We can see if larger engines tend to have better or worse fuel economy.\n\n**Chart 2: Bar Chart of Fuel Efficiency by Cylinders**\n\n* X-axis: number of cylinders (cyl)\n* Y-axis: average miles per gallon (mpg)\n* This chart can help identify the most fuel-efficient engine configuration. We can see if cars with more or fewer cylinders tend to have better fuel economy.\n\nThese charts will provide insights into two important aspects of vehicle performance: fuel efficiency and engine design.'

In [12]:
from langchain_experimental.utilities import PythonREPL

# Create an instance of PythonREPL
repl = PythonREPL()

# Pass the dataframe into the globals dictionary of the PythonREPL instance
repl.globals['df'] = df

In [13]:
from typing import Annotated
from langchain_core.tools import tool

@tool
def python_repl(
    code: Annotated[str, "The python code to execute to generate your chart."]
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

tools = [python_repl]

In [14]:
instructions_template = '''

You are an agent that writes and excutes python code

You have access to a Python abstract REPL, which you can use to execute the python code.

You must write the python code code assuming that the dataframe (stored as df) has already been read.

If you get an error, debug your code and try again.

You might know the answer without running any code, but you should still run the code to get the answer.

If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.

Do not create example dataframes 
'''

In [15]:
base_template = '''

    {instructions_template}

    TOOLS:

    ------

    You have access to the following tools:

    {tools}

    To use a tool, please use the following format:

    ```

    Thought: Do I need to use a tool? Yes

    Action: the action to take, should be one of [{tool_names}]

    Action Input: the input to the action

    Observation: the result of the action

    ```

    When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

    ```

    Thought: Do I need to use a tool? No

    Final Answer: [your response here]

    ```

    Begin!

    Previous conversation history:

    {chat_history}

    New input: {input}

    {agent_scratchpad}
'''

In [16]:
from langchain_core.prompts import PromptTemplate

base_prompt = PromptTemplate(template=base_template, input_variables=['agent_scratchpad', 'input', 'instructions', 'tool_names', 'tools'])
base_prompt = base_prompt.partial(instructions_template=instructions_template) #format the instructions

In [24]:
new_input = '''
Create the data needed for the following chart

CHART 

-------- 
Chart 1: Bar Chart - Top 10 cars

X-axis: model names

Y-axis: carb\n*

You're task is to write code to transform the raw data into the desired data structure in json to be included in the plot.

Only return the data.

Here is the dataset schema:

{schema}

'''.format(schema=metadata['Schema'])

In [25]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(llm, tools, base_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_out = agent_executor.invoke(
    {
        "input": new_input,
        "chat_history": ""
    }
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No

Final Answer:
```python
import pandas as pd

df_top_10_cars = df.nlargest(10, 'mpg')[['model', 'carb']]
df_top_10_cars_json = df_top_10_cars.to_json(orient='records')

print(df_top_10_cars_json)
```
This will output a JSON string containing the top 10 cars by MPG along with their carb count.

> Finished chain.


In [22]:
agent_out['output']

"```python\nimport pandas as pd\n\n# Assuming the dataframe is stored in df and it has a column for winery names and another for points\nwineries_data = df.groupby('Winery Name')['Points'].mean().reset_index()\nwineries_data = wineries_data.nlargest(10, 'Points')\nwineries_data['Average Points'] = wineries_data['Points']\nwineries_data = wineries_data[['Winery Name', 'Average Points']]\nwineries_data = wineries_data.to_json(orient='records')\n\nprint(wineries_data)\n```"

In [26]:
import pandas as pd

df_top_10_cars = df.nlargest(10, 'mpg')[['model', 'carb']]
df_top_10_cars_json = df_top_10_cars.to_json(orient='records')

print(df_top_10_cars_json)

[{"model":"Toyota Corolla","carb":1},{"model":"Fiat 128","carb":1},{"model":"Honda Civic","carb":2},{"model":"Lotus Europa","carb":2},{"model":"Fiat X1-9","carb":1},{"model":"Porsche 914-2","carb":2},{"model":"Merc 240D","carb":2},{"model":"Datsun 710","carb":1},{"model":"Merc 230","carb":2},{"model":"Toyota Corona","carb":1}]
